# Testing the Named Entities Recognition engine of Spacy

### Step 1. Loading Spacy models
***

We install Spacy's language library for the first run. Then we can comment-out the download command. Note that we are loading Spacy's "medium" model.


In [12]:
import re
import pandas as pd
import spacy
import sys
from collections import Counter
import re


## Run to install the language library, then comment-out
## !{sys.executable} -m spacy download en
!{sys.executable} -m spacy download en_core_web_md

nlp = spacy.load('en_core_web_md')
print('Finished loading.')

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
Finished loading.


### Step 2. Pre-processing
***

* Read two files exported from the database which contain the SE Glossary articles definitions and their titles.
* In later versions, the corresponding tables will be directly exported from the database.
* Merge by id and discard records with duplicate titles and/or definitions.



In [13]:
dat1= pd.read_csv("ESTAT_dat_concepts_2021_04_08.csv",sep=";")
dat2= pd.read_csv("ESTAT_dat_link_info_2021_04_08.csv",sep=";")
Gloss_concepts = pd.merge(dat1,dat2,on=['id'])
del(dat1,dat2)

Gloss_concepts = Gloss_concepts[['id','title','definition']]

Gloss_concepts = Gloss_concepts.drop_duplicates(subset=["definition"])
Gloss_concepts = Gloss_concepts.dropna(axis=0,subset=["definition"])
Gloss_concepts = Gloss_concepts.drop_duplicates(subset=["title"])
Gloss_concepts = Gloss_concepts.dropna(axis=0,subset=["title"])

Gloss_concepts.reset_index(drop=True, inplace=True)
Gloss_concepts.head()

,id,title,definition
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page fo...
1,3,Nights spent,A night spent or tourism night ...
2,4,Supply side tourism statistics,Regulation (EU) No 692/2011 of 6 July 2011 e...
3,5,Tourist,"For tourism statistics, a travell..."
4,7,Asia-Europe Meeting (ASEM),"The Asia-Europe Meeting , abbreviated..."


### Step 3. Pre-processing (cont)
*** 

Keep a backup of the original definition and do some data cleansing.

In [14]:
Gloss_concepts['definition_init'] = Gloss_concepts['definition']


Gloss_concepts['definition'] = Gloss_concepts['definition'].apply(lambda x: re.sub('^ +| +$', '',x)) ## remove start and end spaces
Gloss_concepts['definition'] = Gloss_concepts['definition'].apply(lambda x: re.sub(' +', ' ',x)) ## remove more than one spaces
Gloss_concepts['definition'] = Gloss_concepts['definition'].apply(lambda x: re.sub(' , ',', ',x)) ## space-comma-space -> comma-space
Gloss_concepts['definition'] = Gloss_concepts['definition'].apply(lambda x: re.sub(r'[a-zA-Z]+\d+', '.', x))## letters+digits -> .

Gloss_concepts.head()
#len(Gloss_concepts.definition.values.tolist())



,id,title,definition,definition_init
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page for other mean...,See arrival disambiguation page fo...
1,3,Nights spent,A night spent or tourism night (overnight stay...,A night spent or tourism night ...
2,4,Supply side tourism statistics,Regulation (EU) No 692/2011 of 6 July 2011 est...,Regulation (EU) No 692/2011 of 6 July 2011 e...
3,5,Tourist,"For tourism statistics, a traveller is someone...","For tourism statistics, a travell..."
4,7,Asia-Europe Meeting (ASEM),"The Asia-Europe Meeting, abbreviated as ASEM, ...","The Asia-Europe Meeting , abbreviated..."


### Step 4. Apply the NER engine
***

Create columns ORG, GPE, NORP, LOCATION which will hold dictionaries with entities recognized as:
* Organizations;
* Countries, cities, states;
* Nationalities or religious or political groups;
* Non-GPE locations, mountain ranges, bodies of water, respectively. 

In each dictionary in a record, the key is the entity and the values are a list with the token span's *start* index position, the token span's *stop* index position and the count in the definition of the glossary article.

In [15]:
nlp.max_length = 1500000

Gloss_concepts['ORG'] = [dict() for i in range(len(Gloss_concepts))]
Gloss_concepts['GPE'] = [dict() for i in range(len(Gloss_concepts))]
Gloss_concepts['NORP'] = [dict() for i in range(len(Gloss_concepts))]
Gloss_concepts['LOCATION'] = [dict() for i in range(len(Gloss_concepts))]

for i in range(len(Gloss_concepts)):
    if i % 100 == 0: print('i = ',i,' of ',len(Gloss_concepts))
    tokens = nlp(Gloss_concepts.loc[i,'definition'])
    entities = tokens.ents
    for ent in entities:
        #print(ent.text, ent.label_)
        if ent.label_ == 'ORG':
            if ent.text.upper() in Gloss_concepts.loc[i,'ORG'].keys():
                Gloss_concepts.loc[i,'ORG'][ent.text.upper()][0].append((ent.start,ent.end)) 
                Gloss_concepts.loc[i,'ORG'][ent.text.upper()][1] += 1 
            else:    
                Gloss_concepts.loc[i,'ORG'][ent.text.upper()] = [[(ent.start,ent.end)],1]
        
        elif ent.label_ == 'GPE':
            if ent.text.upper() in Gloss_concepts.loc[i,'GPE'].keys():
                Gloss_concepts.loc[i,'GPE'][ent.text.upper()][0].append((ent.start,ent.end)) 
                Gloss_concepts.loc[i,'GPE'][ent.text.upper()][1] += 1 
            else:    
                Gloss_concepts.loc[i,'GPE'][ent.text.upper()] = [[(ent.start,ent.end)],1]
                
        elif ent.label_ == 'NORP':
            if ent.text.upper() in Gloss_concepts.loc[i,'NORP'].keys():
                Gloss_concepts.loc[i,'NORP'][ent.text.upper()][0].append((ent.start,ent.end)) 
                Gloss_concepts.loc[i,'NORP'][ent.text.upper()][1] += 1 
            else:    
                Gloss_concepts.loc[i,'NORP'][ent.text.upper()] = [[(ent.start,ent.end)],1]
                
        elif ent.label_ == 'LOCATION':
            if ent.text.upper() in Gloss_concepts.loc[i,'LOCATION'].keys():
                Gloss_concepts.loc[i,'LOCATION'][ent.text.upper()][0].append((ent.start,ent.end)) 
                Gloss_concepts.loc[i,'LOCATION'][ent.text.upper()][1] += 1 
            else:    
                Gloss_concepts.loc[i,'LOCATION'][ent.text.upper()] = [[(ent.start,ent.end)],1]         
    
Gloss_concepts

#PERSON People, including fictional
#NORP Nationalities or religious or political groups
#FACILITY Buildings, airports, highways, bridges, etc.
#ORGANIZATION Companies, agencies, institutions, etc.
#GPE Countries, cities, states
#LOCATION Non-GPE locations, mountain ranges, bodies of water
#PRODUCT Vehicles, weapons, foods, etc. (Not services)
#EVENT Named hurricanes, battles, wars, sports events, etc.
#WORK OF ART Titles of books, songs, etc.
#LAW Named documents made into laws 
#LANGUAGE Any named language
#The following values are also annotated in a style similar to names:
#DATE Absolute or relative dates or periods
#TIME Times smaller than a day
#PERCENT Percentage (including “%”)
#MONEY Monetary values, including unit
#QUANTITY Measurements, as of weight or distance
#ORDINAL “first”, “second”
#CARDINAL Numerals that do not fall under another typ

i =  0  of  1284
i =  100  of  1284
i =  200  of  1284
i =  300  of  1284
i =  400  of  1284
i =  500  of  1284
i =  600  of  1284
i =  700  of  1284
i =  800  of  1284
i =  900  of  1284
i =  1000  of  1284
i =  1100  of  1284
i =  1200  of  1284


,id,title,definition,definition_init,ORG,GPE,NORP,LOCATION
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page for other mean...,See arrival disambiguation page fo...,"{'EUROPEAN UNION': [[(14, 16)], 1], 'EU': [[(1...",{},{},{}
1,3,Nights spent,A night spent or tourism night (overnight stay...,A night spent or tourism night ...,{},{},{},{}
2,4,Supply side tourism statistics,Regulation (EU) No 692/2011 of 6 July 2011 est...,Regulation (EU) No 692/2011 of 6 July 2011 e...,"{'EU': [[(2, 3)], 1]}",{},"{'EUROPEAN': [[(23, 24)], 1]}",{}
3,5,Tourist,"For tourism statistics, a traveller is someone...","For tourism statistics, a travell...",{},{},{},{}
4,7,Asia-Europe Meeting (ASEM),"The Asia-Europe Meeting, abbreviated as ASEM, ...","The Asia-Europe Meeting , abbreviated...","{'ASEM': [[(8, 9), (38, 39)], 2], 'THE EUROPEA...","{'OCEANIA': [[(29, 30)], 1], 'NORWAY': [[(57, ...",{},{}
...,...,...,...,...,...,...,...,...
1279,2879,Statistical territory,The statistical territory of the European Unio...,The statistical territory of the ...,"{'THE EUROPEAN UNION': [[(4, 7)], 1], 'EU': [[...","{'GERMANY': [[(40, 41)], 1], 'HELIGOLAND': [[(...","{'FRENCH': [[(62, 63)], 1], 'MARTINIQUE': [[(7...",{}
1280,2881,Statistical indicator,A statistical indicator is the representation ...,A statistical indicator is the repres...,"{'EUROPEAN UNION': [[(82, 84)], 1], 'EU': [[(8...",{},{},{}
1281,2885,Tourism industries,For the article Tourism industries - economic ...,For the article Tourism industries - eco...,"{'UNWTO': [[(21, 22)], 1], 'SBS': [[(134, 135)...",{},"{'EUROPEAN': [[(29, 30)], 1]}",{}
1282,2891,Sawnwood,Sawnwood is wood that has been produced either...,Sawnwood is wood that has been produced ...,"{'SAWNWOOD': [[(0, 1)], 1]}",{},{},{}


### Step 5. Gathering the most common entities: example with ORG entities
***

We can see several errors and repetitions. These require some further cleansing steps and fine-tuning of the NER engine (not yet carried out).


In [16]:
from itertools import chain
org_list=sorted(list(chain.from_iterable(Gloss_concepts['ORG'].apply(lambda x: x.keys()))))
org_all_freqs = sorted(Counter(org_list))
print(org_all_freqs)

["'EUROPEAN STATISTICS ON ACCIDENTS AT WORK", '(=', '._MEUR & ._MEUR & ._MEUR & CP_MPPS', '04 FOOD', '08 CHEMICALS', '1999/45/EC', '2007/64/EC', 'A EUROPEAN UNION', 'A UNITED NATIONS', 'A.I.S.B.L', 'AAA', 'AAGR', 'ACL', 'ACP', 'ADDITIVES', 'ADSL', 'AE', 'AEA', 'AEI', 'AES', 'AETIOLOGY', 'AF', 'AGENCY', 'AGRICULTURAL', 'AIC', 'AIR', 'ALL EUROPEAN UNION', 'AN INFORMATION SOCIETY FOR ALL', 'ANGOLA AO EQUATORIAL GUINEA', 'ANIMAL', 'ANNEX I TO COMMISSION DECISION', 'ANNEX IV OF REGULATION 1166/2008', 'ANSER', 'APEC', 'ARA', 'ARAS', 'AROPE', 'ART.2(A', 'ART.2(G', 'ASEAN', 'ASEM', 'ASIA-PACIFIC ECONOMIC COOPERATION', 'ASYLUM', 'ATS', 'AU', 'AUTOREGRESSIVE', 'AWU', 'BALANCE OF PAYMENTS MANUAL', 'BANK', 'BARBUDA AG REPUBLIC', 'BE)BELGIUM', 'BELLADONA, SHAMOTTI', 'BG', 'BGN', 'BIODIVERSITY', 'BIRDS', 'BIS', 'BISON BISON', 'BMI', 'BOARD OF APPEAL', 'BOD', 'BRACKISHWATER CULTURE', 'BRIC', 'BRICS', 'BROADCAST - BROADCASTING', 'BS', 'BULL', 'CAP', 'CC', 'CCP', 'CDMA', 'CENTRAL FRAMEWORK', 'CEPA', "C

In [17]:
len(org_all_freqs)

903

In [18]:

print('\n100 most common:\n')
org_common_freqs = Counter(org_list).most_common(100)
org_common = sorted([x[0] for x in org_common_freqs])
print(org_common_freqs)




100 most common:

[('EU', 211), ('THE EUROPEAN UNION', 139), ('EUROSTAT', 51), ('EUROPEAN UNION', 49), ('EC', 43), ('NACE', 35), ('THE EUROPEAN COMMISSION', 22), ('ESA', 21), ('EFTA', 18), ('ICT', 17), ('THE UNITED NATIONS', 14), ('THE EUROPEAN PARLIAMENT', 13), ('EEC', 11), ('OECD', 11), ('THE EUROPEAN COMMUNITY', 11), ('THE EUROPEAN COUNCIL', 11), ('EEA', 10), ('FSS', 10), ('UN', 10), ('EMU', 8), ('EU-SILC', 8), ('HICP', 8), ('REGULATION', 8), ('STS', 8), ('COMMISSION REGULATION', 7), ('THE EUROPEAN STATISTICAL SYSTEM', 7), ('CO 2', 6), ('COUNCIL REGULATION', 6), ('EHIS', 6), ('FDI', 6), ('ISCED', 6), ('LFS', 6), ('THE EUROPEAN CENTRAL BANK', 6), ('THE EUROPEAN ECONOMIC AREA', 6), ('THE INTERNATIONAL MONETARY FUND', 6), ('BMI', 5), ('COMMISSION', 5), ('DK', 5), ('GNI', 5), ('HRST', 5), ('NUAA', 5), ('PPS', 5), ('S&T', 5), ('SAPM', 5), ('SBS', 5), ('THE COUNCIL OF THE EUROPEAN UNION', 5), ('THE EUROPEAN PATENT OFFICE', 5), ('THE INTERNATIONAL LABOUR ORGANIZATION', 5), ('TPP', 5), ('U

### Step 6. Storing information on these most common entities per article: example with ORG entities
***

This is one way of storing the information on both all entities and counts and on the most common ones in a Pandas dataframe.


In [19]:
Gloss_concepts['ORG_COMMON_100'] = Gloss_concepts['ORG'].apply(lambda x: {y:x[y] for y in x.keys() if y in org_common})
Gloss_concepts

,id,title,definition,definition_init,ORG,GPE,NORP,LOCATION,ORG_COMMON_100
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page for other mean...,See arrival disambiguation page fo...,"{'EUROPEAN UNION': [[(14, 16)], 1], 'EU': [[(1...",{},{},{},"{'EUROPEAN UNION': [[(14, 16)], 1], 'EU': [[(1..."
1,3,Nights spent,A night spent or tourism night (overnight stay...,A night spent or tourism night ...,{},{},{},{},{}
2,4,Supply side tourism statistics,Regulation (EU) No 692/2011 of 6 July 2011 est...,Regulation (EU) No 692/2011 of 6 July 2011 e...,"{'EU': [[(2, 3)], 1]}",{},"{'EUROPEAN': [[(23, 24)], 1]}",{},"{'EU': [[(2, 3)], 1]}"
3,5,Tourist,"For tourism statistics, a traveller is someone...","For tourism statistics, a travell...",{},{},{},{},{}
4,7,Asia-Europe Meeting (ASEM),"The Asia-Europe Meeting, abbreviated as ASEM, ...","The Asia-Europe Meeting , abbreviated...","{'ASEM': [[(8, 9), (38, 39)], 2], 'THE EUROPEA...","{'OCEANIA': [[(29, 30)], 1], 'NORWAY': [[(57, ...",{},{},"{'THE EUROPEAN UNION': [[(41, 44)], 1], 'EU': ..."
...,...,...,...,...,...,...,...,...,...
1279,2879,Statistical territory,The statistical territory of the European Unio...,The statistical territory of the ...,"{'THE EUROPEAN UNION': [[(4, 7)], 1], 'EU': [[...","{'GERMANY': [[(40, 41)], 1], 'HELIGOLAND': [[(...","{'FRENCH': [[(62, 63)], 1], 'MARTINIQUE': [[(7...",{},"{'THE EUROPEAN UNION': [[(4, 7)], 1], 'EU': [[..."
1280,2881,Statistical indicator,A statistical indicator is the representation ...,A statistical indicator is the repres...,"{'EUROPEAN UNION': [[(82, 84)], 1], 'EU': [[(8...",{},{},{},"{'EUROPEAN UNION': [[(82, 84)], 1], 'EU': [[(8..."
1281,2885,Tourism industries,For the article Tourism industries - economic ...,For the article Tourism industries - eco...,"{'UNWTO': [[(21, 22)], 1], 'SBS': [[(134, 135)...",{},"{'EUROPEAN': [[(29, 30)], 1]}",{},"{'SBS': [[(134, 135), (229, 230)], 2], 'STS': ..."
1282,2891,Sawnwood,Sawnwood is wood that has been produced either...,Sawnwood is wood that has been produced ...,"{'SAWNWOOD': [[(0, 1)], 1]}",{},{},{},{}


### Step 7. Exporting the dataframe to Excel
***
This is useful for the manual inspection and the design of rules for the fine-tuning of the NER engine. This output can then directly be imported in the database.


In [20]:
Gloss_concepts.to_excel('Gloss_concepts.xlsx')